# PseudoLabeling Manual

In [1]:
%%capture
!pip install ultralytics

In [2]:

%%capture
!wget https://www.dropbox.com/scl/fi/3mwyjefpr86kqktwrs1dh/HeadBest.pt?rlkey=b1h3rsr1z4cb82yh2najhvmrj&st=k4l2wefe&dl=0
!mv HeadBest.pt?rlkey=b1h3rsr1z4cb82yh2najhvmrj HeadBest.pt

In [3]:
%%capture
!wget https://www.dropbox.com/scl/fi/ejtve3v8cggqo1d7vfmr9/dataSetUnlabel.zip?rlkey=lyun664rceo9shrfie7rh9wsc&st=aldjww59&dl=0
!mv dataSetUnlabel.zip?rlkey=lyun664rceo9shrfie7rh9wsc dataSetUnlabel.zip
!unzip dataSetUnlabel.zip

In [4]:
from ultralytics import YOLO

# Cargar el modelo preentrenado
model = YOLO('HeadBest.pt')


In [5]:
import os
import shutil
import random

# Definir las rutas a los directorios
base_dir = '/content/dataSet'  # Reemplaza esto con la ruta correcta
unlabel_dir = os.path.join(base_dir, 'Unlabel')
train_images_dir = os.path.join(base_dir, 'images', 'train')
train_labels_dir = os.path.join(base_dir, 'labels', 'train')
processed_images_file = os.path.join(base_dir, 'processed_images.txt')

# Crear los directorios de destino si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)

def load_processed_images(processed_images_file):
    if os.path.exists(processed_images_file):
        with open(processed_images_file, 'r') as f:
            return set(line.strip() for line in f)
    return set()

def save_processed_images(processed_images_file, processed_images):
    with open(processed_images_file, 'w') as f:
        for image_path in processed_images:
            f.write(f"{image_path}\n")

def get_unlabeled_images(batch_size=100):
    processed_images = load_processed_images(processed_images_file)
    all_images = [os.path.join(unlabel_dir, f) for f in os.listdir(unlabel_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Filtrar las imágenes ya procesadas y verificar existencia
    unprocessed_images = [img for img in all_images if img not in processed_images and os.path.exists(img)]

    # Seleccionar secuencialmente las primeras `batch_size` imágenes no procesadas
    return unprocessed_images[:batch_size]

def predict_and_label_images(images, model, confidence_threshold=0.7):
    labeled_images = {}

    for image_path in images:
        results = model.predict(image_path, conf=confidence_threshold, verbose=False)

        for result in results:
            boxes = result.boxes
            confs = boxes.conf.cpu().numpy()
            clss = boxes.cls.cpu().numpy()
            data = boxes.xywhn.cpu().numpy()

            for conf, box, cls in zip(confs, data, clss):
                if conf >= confidence_threshold:
                    class_id = int(cls)  # Convertir la clase a entero
                    x_center, y_center, width, height = box[:4]  # Coordenadas de la caja
                    if image_path not in labeled_images:
                        labeled_images[image_path] = []
                    labeled_images[image_path].append((class_id, x_center, y_center, width, height))

    return labeled_images

def save_predictions(predicted_labels, train_images_dir, train_labels_dir, processed_images):
    for image_path, predictions in predicted_labels.items():

        # Generar un nuevo nombre de archivo basado en los existentes en la carpeta de entrenamiento
        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(train_images_dir, image_filename)

        if os.path.exists(new_image_path):
            base_name, ext = os.path.splitext(image_filename)
            counter = 1
            new_image_path = os.path.join(train_images_dir, f"{base_name}_{counter}{ext}")
            while os.path.exists(new_image_path):
                counter += 1
                new_image_path = os.path.join(train_images_dir, f"{base_name}_{counter}{ext}")

        # Mover la imagen al directorio de entrenamiento
        shutil.move(image_path, new_image_path)

        # Guardar las etiquetas en el formato YOLO
        label_path = os.path.join(train_labels_dir, f"{os.path.splitext(image_filename)[0]}.txt")
        with open(label_path, 'w') as f:
            for class_id, x_center, y_center, width, height in predictions:
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

        # Añadir la imagen al conjunto de imágenes procesadas
        processed_images.add(image_path)

def pseudolabeling_iteration(model, num_iterations=5, batch_size=100, confidence_threshold=0.7):
    processed_images = load_processed_images(processed_images_file)

    for i in range(num_iterations):
        print(f"Iteración {i+1}/{num_iterations}")

        # Seleccionar un lote de imágenes sin etiquetar
        unlabeled_images = get_unlabeled_images(batch_size)

        if not unlabeled_images:
            print("No quedan más imágenes sin etiquetar.")
            break

        # Obtener las predicciones para el lote de imágenes
        predicted_labels = predict_and_label_images(unlabeled_images, model, confidence_threshold)

        # Guardar las predicciones y mover las imágenes etiquetadas
        save_predictions(predicted_labels, train_images_dir, train_labels_dir, processed_images)

        # Guardar el estado de las imágenes procesadas
        save_processed_images(processed_images_file, processed_images)

        # Entrenar el Modelo
        model.train(data='dataSet.yaml',
                      epochs=10,
                      #imgsz=250,
                      batch=-1,
                      plots = True)

# Ejecutar el proceso de pseudolabeling
pseudolabeling_iteration(model, num_iterations=5, batch_size=30000, confidence_threshold=0.7)


Iteración 1/5


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=HeadBest.pt, data=dataSet.yaml, epochs=10, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 755k/755k [00:00<00:00, 106MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

...

      10/10         9G      1.107     0.7838      1.681         29        640: 100%|██████████| 29/29 [00:18<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.01it/s]

                   all        109        224      0.695      0.599      0.612      0.255



10 epochs completed in 0.061 hours.
Optimizer stripped from runs/detect/train2222/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train2222/weights/best.pt, 52.0MB

Validating runs/detect/train2222/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.09s/it]


                   all        109        224      0.638      0.643      0.629      0.275
                  Head        107        111      0.672      0.683      0.647      0.295
                  Tail        106        113      0.604      0.602      0.611      0.255
Speed: 0.5ms preprocess, 12.0ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train2222


In [7]:
metrics = model.val(split = 'test', plots = True)
metrics.results_dict

Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/dataSet/labels/test... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<00:00, 953.81it/s]

val: New cache created: /content/dataSet/labels/test.cache



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.06s/it]


                   all         42         85      0.725      0.729      0.706      0.327
                  Head         42         42      0.669      0.714      0.651      0.316
                  Tail         41         43       0.78      0.743      0.762      0.338
Speed: 0.4ms preprocess, 47.9ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/train22222


{'metrics/precision(B)': 0.7248247647390604,
 'metrics/recall(B)': 0.7287540176519232,
 'metrics/mAP50(B)': 0.7063760830562358,
 'metrics/mAP50-95(B)': 0.3268425152228971,
 'fitness': 0.364795872006231}

In [6]:
import shutil
shutil.make_archive('images', 'zip',train_images_dir)
shutil.make_archive('labels', 'zip',train_labels_dir)

'/content/labels.zip'